# Модель выбытия жилищного фонда 

In [145]:
import requests
from cmasf import pandas_sql as pds
import pandas as pd
import sqlalchemy as sa
import re
from os import path, remove
import glob
from bs4 import BeautifulSoup
import datetime as dt

strDB_Temp=path.join('DB', 'Temp') # путь к папке с временными файлами
strBaseDB_URL=r'https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page={page_num}&pageSize={page_size}'

strExportDataURL=r'https://www.reformagkh.ru/opendata/export/{num}'

## База данных - сбор

<p>Источник данных - https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page=1&pageSize=100</p>
<p>Качаем паспорта домов из <b>"Реестра домов"</b> по каждому региону</i> </p>

<div class="alert alert-block alert-info">
<p>Условие попадания дома в расчеты модели: </p>
    <ul>
        <li>ненулевые данные об общей площади жилых помещений <b>(living_rooms_sq > 0)</b></li>
        <li>ненулевое количество жилых помещений <b>(living_rooms_amount > 0)</b></li>
        <li>соотношение этих параметров находилось в интервале между 3 и 1000 <b>(3 < living_rooms_sq/living_rooms_amount < 1000 )</b></li>
    </ul>
</div>

<div class="alert alert-block alert-success">
<p><b>Поля, используемые в расчетах:</b></p>
     <ol>
         <li>Возраст здания <b>(текущий год - commission_year)</b></li>
         <li>Этажность</li>
      </ul>
</div>


In [194]:
re_p_cap_class=re.compile('lh-27 f-18 fw-700') # объект для поиска заголовка строки, например, "...по Алтайскому краю"
# re_p_cap_text=re.compile(r'отчет КР 1\.1') # объект для поиска признака "Капитальный ремонт", это не очень нужно, т.к. данные читаются из соотв. раздела
re_a_export=re.compile(r'/opendata/export/\d+') # объект для поиска признака ссылки на файл csv
re_p_date=re.compile('Актуальная версия:') # объект для поиска даты актиализации

re_cap=re.compile('\sдомов по([А-Яа-я -]+)') # объект для поиска части заголовка - "по Алтайскому краю" - для идентификации ссылки на файл
re_a_num=re.compile('\d+$') # объект для поиска номера (кода, НЕ соотв. коду региона) файла csv
re_date=re.compile('\d{2}\.\d{2}\.\d{4}') # объект для поиска захвата даты актуализации

re_pass_dt=re.compile('p-5 mx-4') # объект для поиска куска с паспортом набора даных

def delete_temp_files(folder=strDB_Temp):
    """Удаление файлов во временой папке - ОСТОРОЖНО!!! УДАЛЯЕТ ВСЕ ФАЙЛЫ В УКАЗАННОЙ ПАПКЕ!!!
    Вызывать для очистки папки временных файлов после обновления базы данных"""
    files = glob.glob(path.join(folder, '*.zip'))
    for f in files:
        remove(f)

def read_data_password():
    """Вспомогательная функция - чтение паспорта набора данных
    читаем структуру данных в файлах csv - считаем, что она одинакова для всех наборов, поэтому читаем и парсим только
    первую. Структура нужна для расшифровки названия полей в файлах и выборки необходимых для модели"""
    rgkh_html=requests.get(strBaseDB_URL.format(page_num=1, page_size=10)) # читаем только первую страницу
    gkh_soup=BeautifulSoup(rgkh_html.text, 'html.parser')
    
    omg=gkh_soup.find('p', {'class':re_p_cap_class}, text=re_cap)
    div_pass=omg.find_next('div', class_=re_pass_dt)
    dv_rows=div_pass.findAll('div', class_='row')

    dct_pass=dict()
    for row in dv_rows:
        dv=row.findAll('div')
        dct_pass.setdefault(dv[0].text, dv[1].text)
    return dct_pass

read_data_password()
print(read_data_password())

# lst_neede_fileds=['subject_rf', 'mun_obr_oktmo', 'mun_obr',  'houseguid', 'address', 'commission_year', 'total_sq', ]

{'id': 'ID дома на Портале', 'region_id': 'Субъект РФ (код ФИАС)', 'area_id': 'Район (код ФИАС)', 'city_id': 'Населенный пункт (код ФИАС)', 'street_id': 'Улица (код ФИАС)', 'shortname_region': 'Тип Субъекта РФ', 'formalname_region': 'Субъект РФ (наименование)', 'shortname_area': 'Тип района', 'formalname_area': 'Район (наименование)', 'shortname_city': 'Тип населенного пункта', 'formalname_city': 'Населенный пункт (наименование)', 'shortname_street': 'Тип улицы', 'formalname_street': 'Улица (наименование)', 'house_number': 'Номер дома', 'building': 'Строение', 'block': 'Корпус', 'letter': 'Литера', 'address': 'Адрес дома', 'houseguid': 'Глобальный уникальный идентификатор дома', 'management_organization_id': 'ID Управляющей организации на Портале', 'built_year': 'Год постройки', 'exploitation_start_year': 'Год ввода в эксплуатацию', 'project_type': 'Серия, тип постройки здания', 'house_type': 'Тип дома', 'is_alarm': 'Факт признания дома аварийным', 'method_of_forming_overhaul_fund': 'С

In [188]:
dctCodeReg=dict()
for o in gkh_soup.findAll('option', {'value':True}):
    dctCodeReg[o['value']]=o.text
# print(dctCodeReg)

In [195]:
# dctCodeReg={o['value']:o.text for o in gkh_soup.findAll('option', {'value':True})}

lst_source=list()

def read_page(page_num=1, page_size=100):
    def parse_row(row):
        a=row.find_all_next('a', {'href':re_a_export}, recursive=False, limit=1)[0]
        pd=row.find_all_next('p', text=re_p_date, recursive=False, limit=1)[0]
        return {'name': re_cap.search(row.text).group(0).strip(), 
                'num': re_a_num.search(a['href']).group(0), 
                'date': dt.datetime.strptime(re_date.search(pd.text).group(0), '%d.%m.%Y') }
    
    rgkh_html=requests.get(strBaseDB_URL.format(page_num=page_num, page_size=page_size))
    gkh_soup=BeautifulSoup(rgkh_html.text, 'html.parser')
    
    rows=[p for p in gkh_soup.findAll('p', {'class':re_p_cap_class}, text=re_cap)]
    if rows:
        lst_source=[parse_row(r) for r in rows]
        return pd.DataFrame(lst_source)
    else:
        return None
for i in range(1, 10):
    dtf=read_page(page_num=i)
    if dtf is not None: 
        lst_source.append(dtf)
    else:
        break
        
pdf=pd.concat(lst_source, ignore_index=True)
pdf

,name,num,date
0,домов по Алтайскому краю,118,2020-07-01
1,домов по Амурской области,167,2020-07-01
2,домов по Архангельской области,171,2020-07-01
3,домов по Астраханской области,91,2020-07-01
4,домов по Белгородской области,170,2020-07-01
...,...,...,...
80,домов по Чеченской Республике,94,2020-07-01
81,домов по Чувашской Республике,93,2020-07-01
82,домов по Чукотскому автономному округу,92,2020-07-01
83,домов по Ямало-Ненецкому автономному округу,106,2020-07-01


In [111]:
for k, v in pdf.iterrows():
    print('read for ', v['name'], strExportDataURL.format(num=v.num), end='...')
    r = requests.get(strExportDataURL.format(num=v.num))
    with open(path.join(strDB_Temp, '{name}.zip'.format(name=v.num)), 'wb') as fz_tmp:
        fz_tmp.write(r.content)
    print('done')

read for  по Алтайскому краю https://www.reformagkh.ru/opendata/export/256...done
read for  по Амурской области https://www.reformagkh.ru/opendata/export/255...done
read for  по Архангельской области https://www.reformagkh.ru/opendata/export/254...done
read for  по Астраханской области https://www.reformagkh.ru/opendata/export/253...done
read for  по Белгородской области https://www.reformagkh.ru/opendata/export/187...done
read for  по Брянской области https://www.reformagkh.ru/opendata/export/186...done
read for  по Владимирской области https://www.reformagkh.ru/opendata/export/185...done
read for  по Волгоградской области https://www.reformagkh.ru/opendata/export/252...done
read for  по Вологодской области https://www.reformagkh.ru/opendata/export/215...done
read for  по Воронежской области https://www.reformagkh.ru/opendata/export/251...done
read for  по городу Москве https://www.reformagkh.ru/opendata/export/184...done
read for  по городу Санкт-Петербургу https://www.reformagkh.ru/

In [146]:
delete_temp_files()

In [112]:
pd_data=pd.read_csv(path.join(strDB_Temp, '172.zip'), sep=';')
pd_data

,subject_rf,mun_obr_oktmo,mun_obr,mkd_code,houseguid,address,commission_year,architectural_monument_status,total_sq,total_rooms_amount,...,money_ppl_collected_date,owners_payment,energy_efficiency,previous_energy_efficiency,energy_audit_date,is_change_energy_efficiency,exclude_date_from_program,comment,last_update,house_id
0,Ямало-Ненецкий автономный округ,71952000,город Губкинский,5200000203,e06d44a7-a40c-496b-a09a-a9b35a8b4974,"г. Губкинский, мкр. 11-й, д. 34",1990,NaN,"827,10",12,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7754544.0
1,Ямало-Ненецкий автономный округ,71952000,город Губкинский,5200000204,7fe8679c-be8d-431a-b25e-10a6e606ab13,"г. Губкинский, мкр. 11-й, д. 35",2004,NaN,"2618,10",18,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7756726.0
2,Ямало-Ненецкий автономный округ,71952000,город Губкинский,5200000208,d83ea86b-5781-45c7-9058-fc5bdebf1452,"г. Губкинский, мкр. 11-й, д. 39",2005,NaN,"2570,30",62,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7756520.0
3,Ямало-Ненецкий автономный округ,71952000,город Губкинский,5200000217,ebbeedc0-e591-43fa-b14e-b6ee79adad28,"г. Губкинский, мкр. 11-й, д. 49",1994,NaN,"809,10",14,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7758341.0
4,Ямало-Ненецкий автономный округ,71952000,город Губкинский,5200000223,175cc98d-e78d-476d-841b-666c29fdda44,"г. Губкинский, мкр. 12-й, д. 1",1997,NaN,"1609,00",18,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7759867.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,Ямало-Ненецкий автономный округ,71928000,Ямальский муниципальный район,2841760127,d29901ce-4ba3-4f24-9296-eef54a87621d,"с. Яр-Сале, ул. Худи Сэроко, д. 45А",2014,NaN,"1520,40",28,...,01.07.2017,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,7546808.0
2399,Ямало-Ненецкий автономный округ,71928000,Ямальский муниципальный район,2841760087,80ebe097-ceb5-48f0-a46d-96884ef3c2b1,"с. Яр-Сале, ул. Худи Сэроко, д. 46",1988,NaN,"336,40",8,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,22.01.2016,NaN,10.06.2020,8454276.0
2400,Ямало-Ненецкий автономный округ,71928000,Ямальский муниципальный район,2841760088,a6cb586d-4316-4902-863e-e69af6d4a980,"с. Яр-Сале, ул. Худи Сэроко, д. 48",2003,NaN,"499,80",6,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,8606954.0
2401,Ямало-Ненецкий автономный округ,71928000,Ямальский муниципальный район,2841760089,7c356fc1-14a4-4a0a-ae09-4311a1172c97,"с. Яр-Сале, ул. Худи Сэроко, д. 49",2009,NaN,"581,90",11,...,01.10.2015,"10,50",не присвоен,NaN,NaN,Нет,NaN,NaN,10.06.2020,9015886.0


In [153]:
dom_html=requests.get(strBaseDB_URL.format(page_num=1, page_size=100))
dsp=BeautifulSoup(dom_html.text, 'html.parser')



5

In [159]:
p_caps=dsp.findAll('p', text=re.compile('Реестр домов по'))
div_p=p_caps[0].find_next('div', class_='row').find('a', href=re_a_export)
print(div_p['href'])

/opendata/export/118
